In [14]:
import pyroomacoustics as pra
import os.path
import speech as sp

In [15]:
snr_vals = [10,20]
desired_word = ['yes','no']
lpc_order = 10
iterations = 5
fft_len = 400
n_fft_bins = fft_len//2 + 1
alpha = 0.05


In [16]:
sub = 2
dataset = pra.datasets.GoogleSpeechCommands(download=True,subset=sub)
noise_samps = dataset.filter(speech=0)
noise = noise_samps[0]
speech_samps = dataset.filter(speech=1)
speech_samps = speech_samps.filter(word=desired_word)

Dataset exists! Using ./google_speech_commands
Dataset exists! Using ./google_speech_commands
Dataset exists! Using ./google_speech_commands
Dataset exists! Using ./google_speech_commands


/home/lcolbois/anaconda3/lib/python3.6/site-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


In [17]:
speech_file_location = {}
for s in speech_samps:
    speech_file_location[s] = s.meta.as_dict()['file_loc']
noise_file_location = noise.meta.as_dict()['file_loc']

In [18]:
dest_dir = 'output'
if not os.path.exists(dest_dir):
    os.makedirs(dest_dir)

In [19]:
for SNR in snr_vals:
    for num,samp in enumerate(speech_samps):
        word = samp.meta.as_dict()['word']
        y_clean,sr = sp.data.load(speech_file_location[samp])
        y = sp.data.add_noise_from_file(y_clean,sr,noise_file_location,SNR)
        s = sp.process.denoise(y,fft_len,lpc_order,iterations)
        s_VAD,_,_ = sp.process.denoise_with_vad(y,sr,fft_len, lpc_order,iterations,alpha)
        id = 'output/'+word+'{}dB_{}'.format(SNR,num)
        sp.data.write(y,sr,id+'_noisy.wav')
        sp.data.write(s,sr,id+'_denoised.wav')
        sp.data.write(s_VAD,sr,id+'_VAD+denoised.wav')

In [37]:
samp = speech_samps[0]

array([ -16,  -10,    2, ..., -160,  -76,    1], dtype=int16)